<a href="https://colab.research.google.com/github/LevProg/NTI_Task/blob/main/MaryBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Проверяем, что включен GPU

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers

Скачиваем данные

In [ ]:
!wget https://russiansuperglue.com/tasks/download/MuSeRC
!mv MuSeRC MuSeRC.zip
!unzip MuSeRC.zip

--2021-01-10 20:07:58--  https://russiansuperglue.com/tasks/download/MuSeRC
Resolving russiansuperglue.com (russiansuperglue.com)... 80.66.93.142
Connecting to russiansuperglue.com (russiansuperglue.com)|80.66.93.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1196720 (1.1M) [application/zip]
MuSeRC: Is a directory

Cannot write to ‘MuSeRC’ (Success).
mv: cannot overwrite non-directory 'MuSeRC.zip' with directory 'MuSeRC'
Archive:  MuSeRC.zip
replace MuSeRC/train.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: MuSeRC/train.jsonl      
replace __MACOSX/MuSeRC/._train.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/MuSeRC/._train.jsonl  
replace MuSeRC/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: MuSeRC/.DS_Store        
replace __MACOSX/MuSeRC/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: __MACOSX/MuSeRC/._.DS_Store  
replace MuSeRC/test.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  

Загружаем данные, предобрабатываем, добавляем токены CLS и SEP

In [ ]:
import json
import re

FEATURES_COUNT = 230

def clean_text(text):

    text = text.lower()
    
    pattern = '\(\d+\)'
    text = re.sub(pattern, '', text)

    text = text.replace('\xad', '')
    text = text.replace('\u00ad', '')
    text = text.replace('\N{SOFT HYPHEN}', '')
    
    text = text.replace('«', '')
    text = text.replace('»', '')  
    text = text.replace('...', '') 
    text = text.replace('—', '') 

    text = re.sub(r'[^\w\s]','',text)

    return text

def text_splitter(text, amount=FEATURES_COUNT): 
    tokens = clean_text(text).split(' ')
    new_text = ' '.join(tokens[-amount:])
    return new_text

def get_X_y_for_bert(data_json_file):
    X, y = [], []
    with open(data_json_file, 'r', encoding='utf8') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    #X.append((text_splitter(text+' textquestionseparator '+query+' '+a['text'])).replace('textquestionseparator', '[SEP]'))
                    X.append('[CLS] '+ (text_splitter(query+' '+a['text']+' textquestionseparator '+text)).replace('textquestionseparator', '[SEP]'))
                    y.append(a['label'])
    return X, y

def get_X(data_json_file):
    X = []
    with open(data_json_file, 'r', encoding='utf8') as json_file:
        json_list = list(json_file)
        #print(json_list[0])
        for json_str in json_list:
            item = json.loads(json_str)
            text = item['passage']['text']
            questions = item['passage']['questions']
            for q in questions:
                query = q['question']
                ans = q['answers']
                for a in ans:
                    X.append('[CLS] '+ (text_splitter(query+' '+a['text']+' textquestionseparator '+text)).replace('textquestionseparator', '[SEP]'))
                    
    return X

In [ ]:
X_train, y_train = get_X_y_for_bert('MuSeRC/train.jsonl')
X_valid, y_valid = get_X_y_for_bert('MuSeRC/val.jsonl')
X_test = get_X('MuSeRC/test.jsonl') 

In [ ]:
X_train[100]

'[CLS] как николай хабибулин принес своей команде победу восьмого ноября в чикаго он поймал все шайбы противника [SEP]  российский вратарь клуба нхл эдмонтон ойлерс николай хабибулин принес своей команде победу в матче с действующим обладателем кубка стэнли чикаго блэк хокс  эта игра состоялась в ночь на 8 ноября в чикаго и завершилась со счетом 21 в пользу эдмонтона  хабибулин отразил 26 из 27 бросков сообщает официальный сайт нхл  единственную шайбу в ворота чикаго в середине первого периода забросил фернандо писани  эдмонтон забил два гола на восьмой минуте третьего периода с интервалом в 14 секунд  отличились крис кертис и сэм ганье  после этой победы у ойлерс стало десять очков в 12 матчах и команда осталась на последнем месте в западной конференции  хабибулин в нынешнем сезоне принял участие в 11 матчах  в среднем он отражает 905 процента бросков по своим воротам и пропускает по 31 шайбы за игру  один матч россиянин отстоял на ноль  из других матчей сыгранных в ночь на 8 ноября с

Меняем предобученный англоязычный берт из примера на мультиязычный (можно попробовать еще русский)     
В берте токен это не только слово, но может и быть часть слова (см. выводы токенайзера далее)   
Если не поменяем модель, то токеном будет каждая буква

In [ ]:
from transformers import BertTokenizer

MODEL_NAME = "bert-base-multilingual-cased"

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
# Print the original sentence.
print(' Original: ', X_train[100])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(X_train[100]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(X_train[100])))

 Original:  [CLS] как николай хабибулин принес своей команде победу восьмого ноября в чикаго он поймал все шайбы противника [SEP]  российский вратарь клуба нхл эдмонтон ойлерс николай хабибулин принес своей команде победу в матче с действующим обладателем кубка стэнли чикаго блэк хокс  эта игра состоялась в ночь на 8 ноября в чикаго и завершилась со счетом 21 в пользу эдмонтона  хабибулин отразил 26 из 27 бросков сообщает официальный сайт нхл  единственную шайбу в ворота чикаго в середине первого периода забросил фернандо писани  эдмонтон забил два гола на восьмой минуте третьего периода с интервалом в 14 секунд  отличились крис кертис и сэм ганье  после этой победы у ойлерс стало десять очков в 12 матчах и команда осталась на последнем месте в западной конференции  хабибулин в нынешнем сезоне принял участие в 11 матчах  в среднем он отражает 905 процента бросков по своим воротам и пропускает по 31 шайбы за игру  один матч россиянин отстоял на ноль  из других матчей сыгранных в ночь на

Максимальная длина одного текста - 512 токенов. Проверяем, что все в порядке:

In [ ]:
max_len = 0

# For every sentence...
for sent in X_train:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  511


In [ ]:
def tokenize_texts(sentences):
  # Tokenize all of the sentences and map the tokens to thier word IDs.
  input_ids = []
  attention_masks = []

  # For every sentence...
  for sent in sentences:
      # `encode_plus` will:
      #   (1) Tokenize the sentence.
      #   (2) Prepend the `[CLS]` token to the start.
      #   (3) Append the `[SEP]` token to the end.
      #   (4) Map tokens to their IDs.
      #   (5) Pad or truncate the sentence to `max_length`
      #   (6) Create attention masks for [PAD] tokens.
      encoded_dict = tokenizer.encode_plus(
                          sent,                      # Sentence to encode.
                          add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                          max_length = 512,           # Pad & truncate all sentences.
                          pad_to_max_length = True,
                          return_attention_mask = True,   # Construct attn. masks.
                          return_tensors = 'pt',     # Return pytorch tensors.
                    )
      
      # Add the encoded sentence to the list.    
      input_ids.append(encoded_dict['input_ids'])
      
      # And its attention mask (simply differentiates padding from non-padding).
      attention_masks.append(encoded_dict['attention_mask'])

  # Convert the lists into tensors.
  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)

  # Print sentence 0, now as a list of IDs.
  print('Original: ', sentences[0])
  print('Token IDs:', input_ids[0])

  return input_ids, attention_masks

In [ ]:
X_train, X_train_masks = tokenize_texts(X_train)
X_valid, X_valid_masks = tokenize_texts(X_valid)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  [CLS] где бегала шпана в парке [SEP]  но люди не могут существовать без природы поэтому в парке стояли железобетонные скамейки  деревянные моментально ломали  в парке бегали ребятишки водилась шпана которая развлекалась игрой в карты пьянкой драками иногда насмерть  имали они тут и девок  верховодил шпаной артемкамыло с вспененной белой головой  людочка сколько ни пыталась усмирить лохмотья на буйной голове артемки ничего у неё не получалось  его кудри издали напоминавшие мыльную пену изблизя оказались что липкие рожки из вокзальной столовой  сварили их бросили комком в пустую тарелку так они слипшиеся неподъёмно и лежали  да и не ради причёски приходил парень к людочке  как только её руки становились занятыми ножницами и расчёской артемка начинал хватать её за разные места  людочка сначала увёртывалась от хватких рук артемки а когда не помогло стукнула его машинкой по голове и пробила до крови пришлось лить йод на голову ухажористого человека  артемка заулюлюкал и со свисто

In [ ]:
X_train[100]

tensor([   101,  10949,  19544,  48613,  35912,    562,  86840, 104037,  12029,
         10913,  38157,  37629,  17117,  10191,  48769,  42320,  10439,  11833,
         64608,  15452,    543,  17891,  10521,  10990,  11060,  10297,  17804,
         10517,  13686,  79703,  98446,  10292,  28471,    102,  28171,  28445,
         11434,  10191,    543,  38790,  28301,  22871,    554,  10353,  10517,
           570,  10746,  67196,  11579,    555,  50751,  45760,  19544,  48613,
         35912,    562,  86840, 104037,  12029,  10913,  38157,  37629,  17117,
         10191,  48769,  42320,    543,  30158,    558,  11323,  24142,  72536,
         23527,  58742,  36069,  25620,  81198,  15888,  16893,  10783,  17891,
         10521,  10990,    542,  10517,  70426,    562,  11899,  10513,  43178,
         28940,  49007,    543,  60092,  10122,    129,  15452,    543,  17891,
         10521,  10990,    549,  10234,  32418,  60603,  11833,  10956,    558,
         54697,  10364,  10296,    543, 

Меняем код под наши переменные. Уменьшаем размер батча, иначе в GPU коллаба не влезает

In [ ]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(X_train, X_train_masks, torch.tensor(y_train))
val_dataset = TensorDataset(X_valid, X_valid_masks, torch.tensor(y_valid))

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 1

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

Дальше алгоритм обучения оставляем, как в примере, только меняем модель

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    #"bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    MODEL_NAME,
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (119547, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  #lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  lr = 5e-7, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 6e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val =11

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        # loss, logits = model(b_input_ids, 
        #                      token_type_ids=None, 
        #                      attention_mask=b_input_mask, 
        #                      labels=b_labels)
        output = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        loss = output.loss
        logits = output.logits


        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.

            # (loss, logits) = model(b_input_ids, 
            #                        token_type_ids=None, 
            #                        attention_mask=b_input_mask,
            #                        labels=b_labels)
            output = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask, 
                           labels=b_labels)
            loss = output.loss
            logits = output.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...
  Batch    40  of  11,950.    Elapsed: 0:00:07.
  Batch    80  of  11,950.    Elapsed: 0:00:14.
  Batch   120  of  11,950.    Elapsed: 0:00:22.
  Batch   160  of  11,950.    Elapsed: 0:00:29.
  Batch   200  of  11,950.    Elapsed: 0:00:36.
  Batch   240  of  11,950.    Elapsed: 0:00:43.
  Batch   280  of  11,950.    Elapsed: 0:00:50.
  Batch   320  of  11,950.    Elapsed: 0:00:57.
  Batch   360  of  11,950.    Elapsed: 0:01:04.
  Batch   400  of  11,950.    Elapsed: 0:01:11.
  Batch   440  of  11,950.    Elapsed: 0:01:18.
  Batch   480  of  11,950.    Elapsed: 0:01:25.
  Batch   520  of  11,950.    Elapsed: 0:01:32.
  Batch   560  of  11,950.    Elapsed: 0:01:39.
  Batch   600  of  11,950.    Elapsed: 0:01:46.
  Batch   640  of  11,950.    Elapsed: 0:01:53.
  Batch   680  of  11,950.    Elapsed: 0:02:00.
  Batch   720  of  11,950.    Elapsed: 0:02:07.
  Batch   760  of  11,950.    Elapsed: 0:02:14.
  Batch   800  of  11,950.    Elapsed: 0:02:2

Строим красивые графики

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

Делаем предсказание и сохраняем файл с submission'ом

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in X_test:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Create the DataLoader.
# prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

In [ ]:
len(predictions)

In [ ]:
pred = np.array([])
for el in predictions:
  pred = np.append(pred, el[:,1], axis=0)

len(pred)

In [ ]:
pred_edited = np.where(pred > 0, 1, 0).tolist()
len(pred_edited)

In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines
def save_output(data, path):
    with jsonlines.open('MuSeRC/test.jsonl') as reader:
        lines = list(reader)
    res = []
    cnt = 0

    for row in lines:
        res_ids = {"idx": row["idx"], "passage": {"questions": []}}

        for line in row["passage"]["questions"]:
          res_line = {"idx": line["idx"], "answers": []}
         
          for answ in line["answers"]:
              res_line["answers"].append({"idx": answ["idx"], "label": int(data[cnt])})
              cnt = cnt + 1
          res_ids["passage"]["questions"].append(res_line)

        res.append(res_ids)


    with open(path, mode="w") as file:
        for line in sorted(res, key=lambda x: int(x.get("idx"))):
            line["idx"] = int(line["idx"])
            file.write(f"{json.dumps(line, ensure_ascii=False)}\n")

In [ ]:
save_output(pred_edited, 'submission.jsonl')

In [ ]:
from google.colab import files
files.download('submission.jsonl') 